## Install Necessary Packages

In [ ]:
!pip install ultralytics roboflow opencv-python pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 

## Import Libraries

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import os
import pandas as pd
import numpy as np
import shutil

ModuleNotFoundError: No module named 'roboflow'

## Download Dataset from Roboflow in YOLOv8 format

**P2_Dhaka_Dataset** is a collection of 3,298 street‐level images from Dhaka,Bangladesh, annotated for eight classes—bicycle, bus, car, CNG, motorcycle,other-vehicle, person, and rickshaw—using YOLO-style normalized bounding boxes.Captured across varied times, angles, and weather conditions.

**Dataset Link:** https://universe.roboflow.com/tanzim-mostafa/p2_dhaka_dataset-f6ba6

**Data Preprocessing & Augmentations in Roboflow:**

**Preprocessing:**

Resize: Stretch to 640x640

**Augmentations:**

Outputs per training example: 5

Flip: Horizontal

Grayscale: Apply to 50% of images

Brightness: Between -25% and +25%

Blur: Up to 1px

In [ ]:
rf = Roboflow(api_key="fCAZwgsZE1ngJoER1svt")
project = rf.workspace("fahad-abdullah-xrq8q").project("p2_dhaka_dataset-f6ba6-uockf")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


## Paths & Model initialization

In [ ]:
data_yaml_path = os.path.join(dataset.location, 'data.yaml')
model = YOLO('yolov8n.pt')

## Train The YOLOv8n Model

### Define Training Parameters

Warm-up with frozen backbone

In [ ]:
warmup_kwargs = {
    'task'        : 'detect',     # object detection
    'data'        : data_yaml_path,
    'epochs'      : 10,           # short warm-up
    'imgsz'       : 640,          # image size
    'batch'       : 8,            # utilize more VRAM
    'device'      : 0,            # GPU index
    'workers'     : 4,            # number of dataloader workers
    'freeze'      : 10,           # lock first 10 layers
    'lr0'         : 0.02,         # initial learning rate
    'lrf'         : 0.1,          # final lr = lr0 * lrf
    'optimizer'   : 'AdamW',      # Adam optimizer
    'weight_decay': 0.0005,
    'amp'         : True,
    'save_period': 1,             # save every epoch
    'project'     : '/content/drive/MyDrive/YOLOv8_walking_assistant_Project',
    'name'        : 'yolov8n_p2dhaka_walking_assistant',
    'exist_ok'    : True
}

print(">> Stage 1: Warm-up (frozen backbone)")
results_warmup = model.train(**warmup_kwargs)

>> Stage 1: Warm-up (frozen backbone)
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/P2_Dhaka_Dataset-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.02, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_p2dhaka_walking_assistant, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=Tr

100%|██████████| 755k/755k [00:00<00:00, 24.7MB/s]

Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,012,408 parameters, 3,012,392 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 'model.3.bn.weight'
Freezing layer 'model.3.bn.bias'
Freezing layer 'model.4.cv1.conv.weight'
Freezing layer 'mode

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      0.99G      1.119     0.8773      1.146         51        640: 100%|██████████| 337/337 [00:45<00:00,  7.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:02<00:00,  6.74it/s]


                   all        301       3052      0.707      0.524      0.594      0.349

10 epochs completed in 0.139 hours.
Optimizer stripped from /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:05<00:00,  3.77it/s]


                   all        301       3052      0.707      0.525      0.594      0.349
               bicycle         69         80      0.503      0.275      0.289      0.121
                   bus        169        271      0.779       0.59      0.662       0.43
                   car        281        852      0.842      0.658      0.755      0.532
                   cng        155        238      0.658      0.623      0.639       0.46
            motorcycle        135        231      0.727      0.483      0.582      0.304
         other-vehicle         69        112        0.7      0.473      0.557      0.261
                person        243        884      0.679      0.529      0.596      0.269
              rickshaw        164        384      0.766      0.568      0.674      0.415
Speed: 0.4ms preprocess, 3.6ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant


Unfreeze & fine-tune full network

In [ ]:
finetune_kwargs = warmup_kwargs.copy()
finetune_kwargs.update({
    'epochs' : 40,     # remaining epochs
    'freeze' : 0,      # unfreeze entire network
    'lr0'    : 0.005,  # lower LR for fine-tuning
})

print("\n>> Stage 2: Fine-tuning (full network)")
results_finetune = model.train(**finetune_kwargs)


>> Stage 2: Fine-tuning (full network)
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/P2_Dhaka_Dataset-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_p2dhaka_walking_assistant, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=

train: Scanning /content/P2_Dhaka_Dataset-1/train/labels.cache... 2694 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2694/2694 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 661.8±455.1 MB/s, size: 89.0 KB)


val: Scanning /content/P2_Dhaka_Dataset-1/valid/labels.cache... 301 images, 0 backgrounds, 0 corrupt: 100%|██████████| 301/301 [00:00<?, ?it/s]


...

      40/40      1.45G     0.8517     0.5918     0.9954         44        640: 100%|██████████| 337/337 [00:51<00:00,  6.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:02<00:00,  8.00it/s]

                   all        301       3052      0.779      0.612      0.685      0.416



40 epochs completed in 0.632 hours.
Optimizer stripped from /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:04<00:00,  3.96it/s]


                   all        301       3052      0.777       0.61      0.683      0.415
               bicycle         69         80      0.604      0.475      0.485      0.227
                   bus        169        271       0.81      0.631      0.711      0.496
                   car        281        852      0.895      0.693      0.814      0.586
                   cng        155        238      0.827      0.689      0.753      0.546
            motorcycle        135        231      0.761       0.58      0.681      0.349
         other-vehicle         69        112      0.839      0.625      0.669      0.337
                person        243        884      0.711      0.521      0.608      0.289
              rickshaw        164        384      0.768      0.663      0.744      0.495
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant


## Inspect Metrics

In [ ]:
def summarize(metrics, stage_name):
    # overall precision and recall
    p     = metrics.box.mp       # mean precision across classes
    r     = metrics.box.mr       # mean recall across classes
    m50   = metrics.box.map50    # mAP@0.5
    m5095 = metrics.box.map      # mAP@0.5:0.95

    print(f"{stage_name} Metrics:")
    print(f"  Precision (P) : {p:.3f}")
    print(f"  Recall    (R) : {r:.3f}")
    print(f"  mAP@50        : {m50:.3f}")
    print(f"  mAP@50:95     : {m5095:.3f}\n")

# Summarize both stages
summarize(results_warmup, "Stage 1 Warm-up")
summarize(results_finetune, "Stage 2 Fine-tune")

Stage 1 Warm-up Metrics:
  Precision (P) : 0.707
  Recall    (R) : 0.525
  mAP@50        : 0.594
  mAP@50:95     : 0.349

Stage 2 Fine-tune Metrics:
  Precision (P) : 0.777
  Recall    (R) : 0.610
  mAP@50        : 0.683
  mAP@50:95     : 0.415



In [ ]:
# Validate the model
metrics = model.val()

# Print the evaluation metrics
print("Model Evaluation Metrics:", metrics)

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1699.7±470.9 MB/s, size: 67.6 KB)


val: Scanning /content/P2_Dhaka_Dataset-1/valid/labels.cache... 301 images, 0 backgrounds, 0 corrupt: 100%|██████████| 301/301 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:04<00:00,  7.64it/s]


                   all        301       3052      0.778      0.609      0.682      0.415
               bicycle         69         80      0.608      0.475      0.477      0.227
                   bus        169        271      0.812      0.631      0.711      0.497
                   car        281        852      0.895      0.692      0.813      0.586
                   cng        155        238      0.829      0.692      0.753      0.546
            motorcycle        135        231      0.752      0.577      0.678      0.348
         other-vehicle         69        112      0.841      0.625      0.668      0.337
                person        243        884      0.719      0.521      0.609      0.288
              rickshaw        164        384      0.772      0.661      0.743      0.493
Speed: 0.5ms preprocess, 5.7ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to /content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant
Model Eval

## Export The Trained Model

In [ ]:
run_dir    = "/content/drive/MyDrive/YOLOv8_Projects/yolov8n_p2dhaka_walking_assistant"
src        = os.path.join(run_dir, "weights", "best.pt")
dst_folder = "/content/drive/MyDrive/YOLOv8_Projects/saved_models"
os.makedirs(dst_folder, exist_ok=True)
dst        = os.path.join(dst_folder, "p2_dhaka_yolov8n_best.pt")

shutil.copy(src, dst)

NameError: name 'os' is not defined

## Export the model in ONNX

In [ ]:
model.export(format='onnx')

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 8.4s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.61...
ONNX: export success ✅ 9.8s, saved as '/content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.onnx' (11.6 MB)

Export complete (10.2s)
Results saved to /conten

'/content/drive/MyDrive/YOLOv8_walking_assistant_Project/yolov8n_p2dhaka_walking_assistant/weights/best.onnx'

In [ ]:
# Show Confusion Matrix
from IPython.display import Image, display
display(Image('/content/runs/detect/train/confusion_matrix.png'))


FileNotFoundError: No such file or directory: '/content/runs/detect/train/confusion_matrix.png'

FileNotFoundError: No such file or directory: '/content/runs/detect/train/confusion_matrix.png'

<IPython.core.display.Image object>

In [ ]:
# Show Results (image)
display(Image('/content/runs/detect/train/results.png'))


In [ ]:
# Show results (table)
import pandas as pd

# Load the CSV file into a pandas DataFrame
csv_file_path = '/content/runs/detect/train/results.csv'
df = pd.read_csv(csv_file_path)

# Display the DataFrame
print(df)


In [ ]:
# Package imports and train path
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

train_path = '/content/runs/detect/train'


In [ ]:
# Show curves of F1-Confident, Precision-Recall, Precision-Confident and Recall-Confident

# Load your images using OpenCV
img1 = cv2.imread(f'{train_path}/F1_curve.png')
img2 = cv2.imread(f'{train_path}/PR_curve.png')
img3 = cv2.imread(f'{train_path}/P_curve.png')
img4 = cv2.imread(f'{train_path}/R_curve.png')

# Concatenate images horizontally
top_row = cv2.hconcat([img1, img2])
bottom_row = cv2.hconcat([img3, img4])

# Concatenate the two rows vertically
grid = cv2.vconcat([top_row, bottom_row])

# Display the concatenated image
cv2_imshow(grid)


In [ ]:
# Validation Results: Ground Truth vs Prediction

# Load your images using matplotlib's imread
img1 = mpimg.imread(f'{train_path}/val_batch0_labels.jpg')
img2 = mpimg.imread(f'{train_path}/val_batch0_pred.jpg')
img3 = mpimg.imread(f'{train_path}/val_batch1_labels.jpg')
img4 = mpimg.imread(f'{train_path}/val_batch1_pred.jpg')
img5 = mpimg.imread(f'{train_path}/val_batch2_labels.jpg')
img6 = mpimg.imread(f'{train_path}/val_batch2_pred.jpg')

# Get image dimensions (assuming all images are of the same size)
img_height, img_width, _ = img1.shape

# Create a figure with subplots, adjusting the figure size to match the image size
fig, axes = plt.subplots(6, 1, figsize=(img_width / 100, 6 * img_height / 100))  # 6 rows, 1 column

# Add a title to the figure with a bigger font size
# fig.suptitle('Validation Results: Ground Truth vs Prediction', fontsize=24)

# Plot the images and add larger titles for each subplot
axes[0].imshow(img1)
axes[0].set_title("Ground Truth - Batch 0", fontsize=22)  # Larger subplot title
axes[1].imshow(img2)
axes[1].set_title("Prediction - Batch 0", fontsize=22)

axes[2].imshow(img3)
axes[2].set_title("Ground Truth - Batch 1", fontsize=22)
axes[3].imshow(img4)
axes[3].set_title("Prediction - Batch 1", fontsize=22)

axes[4].imshow(img5)
axes[4].set_title("Ground Truth - Batch 2", fontsize=22)
axes[5].imshow(img6)
axes[5].set_title("Prediction - Batch 2", fontsize=22)

# Turn off axes for all subplots
for ax in axes:
    ax.axis('off')

# Adjust layout to prevent overlap and show the title
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Display the plot
plt.show()


In [ ]:
# Predict on test data using trained model

# Load trained model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Specify the test folder path
test_path = '/content/animals-2/test/images/'

# List all images in the test folder
image_files = [os.path.join(test_path, img) for img in os.listdir(test_path) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Predict each images
for img_path in image_files:
    results = model.predict(source=img_path, save=True, conf=0.50)


In [ ]:
# Plot prediction on test images

# Specify the predict folder path
pred_path = '/content/runs/detect/predict'

# List all images in the predict folder
pred_image_files = [os.path.join(pred_path, img) for img in os.listdir(pred_path) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Number of images per row
images_per_row = 5

# Calculate the number of rows needed
n_rows = len(pred_image_files) // images_per_row + int(len(pred_image_files) % images_per_row != 0)

# Set figure size
fig, axs = plt.subplots(n_rows, images_per_row, figsize=(15, 3 * n_rows))

# Flatten axes if necessary (for easier iteration)
axs = axs.flatten()

# Loop through each image and display it
for i, img_path in enumerate(pred_image_files):
    img = mpimg.imread(img_path)      # Read the image
    axs[i].imshow(img)                # Show the image
    axs[i].axis('off')                # Turn off axis
    axs[i].set_title(f"Image {i+1}")  # Add title

# Hide any extra empty subplots (if the number of images is not a perfect multiple of images_per_row)
for j in range(i+1, len(axs)):
    axs[j].axis('off')  # Hide unused axes

# Display all images
plt.tight_layout()
plt.show()


In [ ]:
# The accuracy of the model is very low because it has been trained over very few epochs.
# This project is only for learning purpose.
